In [ ]:
import pandas as pd
import numpy as np
import cupy as cp
import nbimporter
import util_func
import time
import tensorflow as tf
from itertools import product

In [ ]:
class FE_Data_M:
    def __init__(self, beta, data_path, BC_mask):
        self.beta = beta
        self.data_path = data_path
        self.BC_mask = BC_mask
        
        snap = pd.read_csv(r"{path}_Snapshots.csv".format(path=data_path), index_col=0)
        
        self.beta_mu = []
        self.times_mu = []
        self.Uc_mu = []
        self.Fc_mu = []
        for i, mu in enumerate(product(beta[0], beta[1], beta[2])):
            snap_subset = snap[(snap['beta_d'] == mu[0]) & (snap['beta_m'] == mu[1]) & (snap['beta_p'] == mu[2])]
            #status_subset = status[(status['beta_d'] == mu[0]) & (status['beta_m'] == mu[1]) & (status['beta_p'] == mu[2])]
            
            self.beta_mu.append(snap_subset.iloc[:, :3].to_numpy())
            
            self.times_mu.append(snap_subset.iloc[:, 3].to_numpy())

            self.Uc_mu.append(snap_subset.iloc[:, 4:].to_numpy().T[BC_mask,:])

            self.Fc_mu.append(pd.read_csv(r"{path}_Fc_{b_d}_{b_m}_{b_p}.csv"
                                       .format(path=data_path,b_d=mu[0],b_m=mu[1],b_p=mu[2]), index_col=0).to_numpy())
        
        
    
    def get_POD_basis(self, A):
        covariance = np.matmul(np.transpose(A),A)

        L, V = np.linalg.eig(covariance)
        Lambda = np.sort(np.real(L))[::-1]
        L_arg = np.argsort(np.real(L))[::-1]

        phi = np.real(V)[:, L_arg]

        sigma = np.sqrt(Lambda)
        sigma = sigma[~np.isnan(sigma)]
    #     sigma = tf.where(tf.math.is_nan(sigma), tf.zeros_like(sigma), sigma)
        sigma_inv = np.diag(sigma**-1)

        psi = np.matmul(np.matmul(A, phi[:, :sigma.shape[0]]), sigma_inv)

        return sigma, psi
    
     
    def get_K_red_0(self, psi, l):
        
        combination = self.beta.shape[1]**self.beta.shape[0]
        K_red_0 = [[None]*combination for i in range(l)]

        for j, mu in enumerate(product(self.beta[0], self.beta[1], self.beta[2])):
            Kc_0 = cp.asarray(pd.read_csv(r"{path}_Kc0_{b_d}_{b_m}_{b_p}.csv"
                                .format(path=self.data_path,b_d=mu[0],b_m=mu[1],b_p=mu[2]), index_col=0).to_numpy())

            print("checkpoint {ch}".format(ch=j))

            for i in range(l):
                psi_l = psi[:,:i+1]
                K_red_0[i][j] = cp.matmul(cp.matmul(cp.transpose(psi_l),Kc_0), psi_l)
                
        return K_red_0
            